In [ ]:
from typing import List

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

import plotly.graph_objs as go
import plotly.plotly as py

from flask_cors import CORS

import pandas as pd

import os
import requests
import io

url='https://raw.githubusercontent.com/dkremlg/Booking-Curves/master/in_GUI.csv'
s=requests.get(url).content
df=pd.read_csv(io.StringIO(s.decode('utf-8')),sep=',')

df['Dprio']=df['Dprio'].astype('float')
df['NumPax']=df['NumPax'].astype('float')

app = dash.Dash(__name__)
app.css.append_css({'external_url': 'https://cdn.rawgit.com/plotly/dash-app-stylesheets/2d266c578d2a6e8850ebce48fdb52759b2aef506/stylesheet-oil-and-gas.css'})  # noqa: E501
server = app.server
CORS(server)

colors = {
    'background': '#111111',
    'text': '#7FDBFF'
}

app.layout = html.Div(children=[html.H1(children='Booking Curve Analytics', style={
            'textAlign': 'center'
            # ,'color': colors['text']
        }),
        html.H3(children='Case Study: Flights with strategic fares', style={
        'textAlign': 'left'
        # ,'color': colors['text']
                                })
    ,
    dcc.Dropdown(id='weekday-dropdown', options=[
        #{'label': i, 'value': i} for i in df['dday'].unique().tolist()
        {'label': i, 'value': i} for i in ['Monday', 'Tuesday', 'Thursday', 'Friday']
    ], multi=False, placeholder='Filter by weekday...'),
    dcc.Graph(id='graph-with-markdown')
    ])

line_columns = ['forecast', 'NumPax', 'PI_lb', 'PI_ub']
linecolor_dict = dict(zip(line_columns,['blue', 'orange', 'aquamarine', 'aquamarine']))
linedash_dict = dict(zip(line_columns,['solid', 'solid', 'dot', 'dot']))
name_dict = dict(zip(line_columns,['Forecast', 'Actual', 'PI Lower', 'PI Upper']))

@app.callback(
    Output('graph-with-markdown', 'figure'),
    [Input('weekday-dropdown', 'value')])
def update_figure(selected_day):
    filtered_df = df.loc[df['dday'] == selected_day,:]
    traces = []
    for i in ['forecast','NumPax','PI_lb','PI_ub']:
        traces.append(go.Scatter(
            x=filtered_df['Dprio'].tolist(),
            y=filtered_df[i].tolist(),
            mode = 'lines',
            name = name_dict[i],
            line = dict(
              dash = linedash_dict[i],
              color = linecolor_dict[i],
              width = 2
           )
))

    return {
        'data': traces,
        'layout': go.Layout(
            xaxis={'title': 'Days before departure', 'range': [0, 365]},
            yaxis={'title': 'Number of bookings', 'range': [0, filtered_df['PI_ub'].max()]})
    }

if __name__ == '__main__':
    app.run_server(debug=True,port=3096)